# Guide: Setting Up and Testing AWS RDS with PySpark

## 1. Create AWS RDS Instance
- Launch a new PostgreSQL RDS instance in AWS.  
- Configure the **security group rules** to allow inbound connections (e.g., port 5432) from your local machine’s IP address.  

## 2. Connect RDS to Local pgAdmin
- Open **pgAdmin** on your local machine.  
- Create a new server connection using your RDS endpoint, port, database name, username, and password.  

## 3. Set Up PostgreSQL Schema
- Once connected, create a new schema for testing:  

```sql
CREATE SCHEMA schema_pyspark;
```

## 4. Test Initial Query
- Try running the following query:  

```sql
SELECT * FROM schema_pyspark."WordCount";
```

- At this stage, you will receive an error because the `WordCount` table does not yet exist.  

## 5. Load Data with PySpark Notebook
- Run the provided Jupyter Notebook: **`loadTORDSWordDataDemo.ipynb`**.  
- This notebook will write the WordCount results into the RDS schema.  

## 6. Verify Results
- Re-run the query:  

```sql
SELECT * FROM schema_pyspark."WordCount";
```

- You should now see the results populated by your PySpark job.  


In [4]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, explode
import pyspark.sql.functions as f

# COMMAND ----------

# Extract
spark = SparkSession.builder.appName("ETL Pipeline").config("spark.jars", "postgresql-42.6.2.jar").getOrCreate()
df = spark.read.text("WordData.txt")

# COMMAND ----------

# Transformation
df2 = df.withColumn("splitedData", f.split("value"," "))
df3 = df2.withColumn("words", explode("splitedData"))
wordsDF = df3.select("words")
wordCount = wordsDF.groupBy("words").count()

print(wordCount.show())



+-----------+-----+
|      words|count|
+-----------+-----+
|   Tomorrow|    4|
|         If|    8|
|      leave|    4|
|      corny|    4|
|        day|    4|
|preoccupied|    4|
|       even|    8|
|      crazy|    4|
|    bananas|    4|
|     priest|    4|
|        did|    4|
|    whether|    4|
|     Having|    4|
|        I'm|    4|
|      crime|    4|
|  surprised|    4|
|      James|    4|
|      could|    8|
|        buy|    4|
|        him|    8|
+-----------+-----+
only showing top 20 rows
None


In [5]:
# COMMAND ----------

# Load

driver = "org.postgresql.Driver"
#url = "rds001.czcasq2eoak3.ap-southeast-1.rds.amazonaws.com"
url = "jdbc:postgresql://rds001.czcasq2eoak3.ap-southeast-1.rds.amazonaws.com/"
table = "schema_pyspark.WordCount"
user = "postgres"
password = "lyx1998225"

wordCount.write.format("jdbc").option("driver", driver).option("url",url).option("dbtable", table).option("mode", 
"append").option("user",user).option("password", password).save()


AnalysisException: Table or view 'schema_pyspark.WordCount' already exists. SaveMode: ErrorIfExists.